In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time
import re
from progressbar import ProgressBar
from dateutil.parser import parse
import datetime

# load data

In [4]:
import twint
import nest_asyncio
d_begin = "2022-03-10"
d_end = "2022-04-02"
daily_datarange = pd.date_range(start=d_begin,
                                end=d_end ,
                                freq='b')

nest_asyncio.apply()
tweet_ids = []
tweet_urls  = []

for start_i in range(0, len(daily_datarange), 1):
    c = twint.Config()
    c.Username = "httweets" 
    c.Pandas = True
    c.Since = str(daily_datarange[start_i].strftime("%Y-%m-%d"))
    c.Until = str((daily_datarange[start_i] + datetime.timedelta(days=2)).strftime("%Y-%m-%d"))
    c.Store_csv = True
    c.Custom_csv = ["id", "user_id", "username", "tweet"]
    c.Output = ("C:/Users/13a71/Dropbox/github/crawler/tweeter crawler/twint/htt/{}.csv".format(c.Since))
    c.Filter_retweets = True
    # Run
    twint.run.Search(c)
    
    
    tweet_urls.extend([str(tweet.urls) for tweet in tweet_ids])


1502431676789571589 2022-03-12 08:50:00 +0900 <htTweets> Prime Miniser Narendra Modi, who is on a two-day visit in his home state, met his mother at her residence   #ElectionResults #PMModi  https://t.co/RksxjQny7R
1502429160207273985 2022-03-12 08:40:00 +0900 <htTweets> #UkraineRussiaWar | Russian President Vladimir Putin on March 11 said there were some "positive shifts" in talks between #Russia and #Ukraine    https://t.co/LEdOpUrmtX
1502426645210275842 2022-03-12 08:30:00 +0900 <htTweets> India has ordered a high-level inquiry into a missile which landed in Mian Channu city of #Pakistan's Punjab province on March 9    https://t.co/N1ZspORk6e
1502424127482781703 2022-03-12 08:20:00 +0900 <htTweets> Indian students in #Russia concerned about disruption of flights and banking services can consider returning home though there are “no security reasons” for them to leave, the Indian embassy in Moscow has said.  (@Rezhasan reports) #UkraineRussiaWar   https://t.co/0eGCMSLT3Z
1502421610493

In [8]:
searchlist = pd.read_excel('~/documents/htt_raw_tweet0402.xlsx').urls

# load module

In [9]:
#Open/reset progressbar and dataframes
pbar = ProgressBar()

#Create dataframe for storing article contents
content_df = pd.DataFrame(columns=['date', 'title','category', 'author', 'content', 'url'])

#Create dataframe for errorlist
error_df = pd.DataFrame(columns=['url'])

#Set headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

In [10]:
import re
from dateutil.parser import parse
from bs4 import NavigableString

def get_soup(url, headers=None):
    """
    Arguments
    ---------
    url : str
        Web page url
    headers : dict
        Headers for requests. If None, use Mozilla/5.0 as default user-agent

    Returns
    -------
    soup : bs4.BeautifulSoup
        Soup format web page
    """

    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    r = requests.get(url, headers=headers)
    html = r.text
    page = BeautifulSoup(html, 'lxml')
    return page

doublespace_pattern = re.compile('\s+')
lineseparator_pattern = re.compile('\n+')

def normalize_text(text):
    text = text.replace('\t', ' ')
    text = text.replace('\r', ' ')
    text = lineseparator_pattern.sub('\n', text)
    text = doublespace_pattern.sub(' ', text)
    return text.strip()

def to_string(instance):
    final_string = ''
    if isinstance(instance, NavigableString):
        return instance
    for contents in instance.contents:
        final_string += to_string(contents)
    return final_string

def parse_article(url):
    
    """
    Argument
    --------
    url : str
        Web page url

    Returns
    -------
    json_object : dict
    JSON format web page contents
    It consists of
        date : publication date of article
        title : article title
        subtitle : none
        content : text 
        category : World
        source : Jakarta Times
        url : web page url
        scrap_time : scrapped time
    """
    
    soup = get_soup(url)
    title = soup.find('h1').text
    try:
        subtitle = soup.find('h2').text
    except:
        subtitle=''
    
    try:
        date_text = soup.find('div', class_='whowhen').find_all('span')[1].text[5:-5]
        date = parse(date_text).strftime("%Y-%m-%d")
    except:
        date_text = soup.find('div', class_='dateTime').text[9:-4]
        date = parse(date_text[1:]).strftime("%Y-%m-%d")
    sub_content = []
    try:
        for a in soup.find('div', class_ = 'storyDetails').find_all('p'):
            if a.a != None:
                a.a.decompose()
            if a.div != None:
                a.div.decompose()
            if a.strong != None:
                a.strong.decompose()
            sub_content.append(a.text)
    except:
        for a in soup.find('div', class_ = 'article').find_all('p'):
            if a.a != None:
                a.a.decompose()
            if a.div != None:
                a.div.decompose()
            if a.strong != None:
                a.strong.decompose()
            sub_content.append(a.text)
    content = ' '.join(' '.join(sub_content).replace('\n','').replace('\xa0','').replace('\\','').split())
    
    json_object = {
          'date': date,
          'title': title,
          'subtitle': subtitle,
          'content': content,
          'category': 'World',
          'source': 'Hindustan Times',
          'url': url
      }
    return json_object

In [11]:
def save(json_obj, directory):
    date = json_obj.get('date', '')
    title = json_obj.get('title', '')
    filepath = '{}/{}_{}.json'.format(directory, date, re.sub('[^a-zA-Z ]+',"", title[:50]).replace(" ", ""))
    with open(filepath, 'w', encoding='utf-8') as fp:
        json.dump(json_obj, fp, indent=2, ensure_ascii=False)
        print('scraped {}'.format(json_obj['title'][:10]))
        
## CNN SCRAPER
import json
directory = 'C:/Users/13a71/Documents/crawling output/hindu_outcome'
    
    
for url in pbar(searchlist):
    try:
        url = url[2:-2]
        article = parse_article(url)
        save(article, directory)
    except:
        print("this url is error")

scraped ISKCON tem
scraped 'Deeds lik
scraped World Happ
scraped From ‘rive
scraped 'Many fals
scraped 
         
scraped India flew
scraped Lessons fr
scraped Vivek Agni
scraped Gen Bajwa 
scraped What weapo
scraped Vivek Agni
scraped Moderna se
scraped Defects po
scraped This signb
scraped This islan
scraped 'Urge Indi
scraped India to b


scraped Shiv Sena 
scraped 'What's wr
scraped 'Goons sti
scraped Indore man
scraped 'Old photo
scraped Dog sittin
scraped Yuzvendra 
scraped 'Special m
scraped How Pakist
scraped India's oi
scraped In new Pun
scraped Kashmiri P
this url is error
scraped Amit Shah 
scraped 10 ministe
scraped Shiv Sena'
scraped Russian No
scraped Old man in
scraped ‘Somewhat 


scraped Delhi is w
scraped Sikh man’s
scraped Working wi
scraped Foreign mi
scraped G-23 leade
scraped Puneeth wa
scraped Haryana pa
scraped Suvendu hi
scraped Man calls 
scraped Infant fou
scraped Man shower
scraped Hot poles:
scraped Alibaba in
scraped Hot poles:
scraped Why Nargol
scraped Mid-March 
scraped Video| US 
scraped Mumbai cou
scraped Russia-Ukr


scraped China plan
scraped Indian-Ame
scraped '220 new a
scraped ‘Why did N
scraped 'Assure he
scraped Leopard sp
scraped In UP, 4 c
scraped 'Had PM Mo
scraped CLAT 2022:
scraped One black 
scraped Birbhum vi
scraped China plan
scraped Pushkar Si
scraped Hero MotoC
scraped Birbhum vi
scraped Shaheed Di
scraped Why Putin'
scraped From Andy 
scraped Foreign se


scraped PM announc
scraped Vessel jou
scraped Putin scar
scraped Centre rev
scraped As India a
scraped China laun
scraped 10-minute 
scraped At UN, a d
scraped Local TMC 
scraped Exports to
scraped Disaster A
scraped Explained:
scraped Dog gently
scraped Autistic g
scraped Understand
scraped Delhi horr
scraped $1.5bn goa
scraped Woman ‘Kat
scraped Parliament


...

scraped NEET PG Co
scraped Petrol, di
scraped NIT, Delhi
scraped Sri Lanka 
scraped Arvind Kej
scraped Prabhakar 
scraped Open borde
scraped UKPSC CSE 
scraped Shashi Tha
scraped Karnataka 
scraped Afternoon 
scraped How MP chi
scraped 1 million 
this url is error
scraped Yuvraj Sin
scraped Nation's i
scraped Shanghai s
scraped BPSC MVI A
scraped Biden blam


scraped Odisha rap
scraped Controvers
this url is error
scraped 'Sri Lanka
scraped Recruitmen
scraped NTRO Recru
scraped India, Aus
scraped Imran Khan
scraped Russian tr
scraped GST collec
scraped Stranded U
scraped DRDO CEPTA
scraped Villagers 
scraped Did Shashi
scraped Imran’s an
scraped Dad’s reac
scraped Morning br
scraped Woman aske


scraped US imposes
scraped In drugs c
scraped China tell
scraped A Ukraine 
scraped Another su
scraped After unre
scraped Odisha tee
scraped Kalicharan
scraped Covaxin pr
scraped Russia hea
scraped Doctor’s s
scraped Working on
scraped SC rejects
scraped GST collec
scraped US man lin
scraped Physics, M
scraped Over 3,000
this url is error
scraped HT This Da
scraped Russia-Ukr
scraped Sri Lanka 


scraped Breaking: 
scraped Eye on 202
scraped ‘Move to H
scraped Armed cops
scraped Akhilesh Y
scraped PM Modi, N
scraped Modi-Deuba
scraped 'Ker' fami
scraped Noida, Gre
scraped Ramadan ki


scraped ‘Delhi, Pu


In [12]:
content_df

,date,title,category,author,content,url


In [ ]:
content_df1 = content_df.loc[content_df["date"] >= "2021-11-12"].reset_index(drop=True)
content_df1 = content_df1.loc[content_df1["date"] <= "2021-08-22"].reset_index(drop=True)

In [13]:
a = content_df.date.unique()

In [14]:
a

array([], dtype=object)

In [15]:
content_df.dropna(subset = ["date"], inplace=True)
content_df = content_df.drop_duplicates(subset='title')

In [16]:

content_df.to_excel('htt.xlsx')
